In [ ]:
import pandas as pd
! pip install rdkit-pypi
!cat  /etc/lsb-release
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True
from rdkit.Chem import Draw
from IPython.display import display
from rdkit.Chem import rdDistGeom
try:
  import py3Dmol
except:
  !pip install py3Dmol
  import py3Dmol
import py3Dmol

In [ ]:
# importing data + pre-selection
inhib_all=pd.read_csv('pilot_library.csv')
inhib_tri=inhib_all[inhib_all ['zinc'].isna()==False]
inhib_tri=inhib_tri[inhib_tri['logp']<4]
inhib_tri=inhib_tri[inhib_tri['mw']<=470]
inhib_tri=inhib_tri[inhib_tri['mw']>=430]
inhib_tri.to_csv('curated_library')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

show(smi, style='stick'):

takes SMILES as input
and gives 3D conformation as output

from github : https://gist.github.com/korakot/98f0bcd92fdcb8a3b8360ceddd6a5332


In [9]:
def show(smi, style='stick'):
    mol = Chem.MolFromSmiles(smi)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    AllChem.MMFFOptimizeMolecule(mol, maxIters=200)
    mblock = Chem.MolToMolBlock(mol)

    view = py3Dmol.view(width=200, height=200)
    view.addModel(mblock, 'mol')
    view.setStyle({style:{}})
    view.zoomTo()
    view.show()
    Chem.rdmolfiles.MolToPDBFile(mol, "output.pdb")

# example
#show(formula)

Similar as 'show' function

'show_n_conformers' function takes SMILES, formula and number of conformers n as input and gives a pdb files for each 3D conformers as output

Automatically saves the conformers as pdb files in google drive folder of choice

Original function from github : https://greglandrum.github.io/rdkit-blog/posts/2023-02-04-working-with-conformers.html
(Modified to fit our needs)

In [6]:
import shutil
def show_n_conformers(smi, formula, n, style='stick'): # str, str, int
    mol = Chem.MolFromSmiles(smi)
    mol = Chem.AddHs(mol)
    rdDistGeom.EmbedMultipleConfs(mol,n, randomSeed=0xf00d) # <- generate n conformers
    print(mol.GetNumConformers(),mol.GetConformer().Is3D())
    for i in range(n):
      mblock = Chem.MolToMolBlock(mol, confId=i)
      view = py3Dmol.view()
      view.addModel(mblock, 'mol')
      view.setStyle({style:{}})
      view.zoomTo()
      view.show()
      filename = formula+"_confo_"+str(i)+".pdb"
      Chem.rdmolfiles.MolToPDBFile(mol, filename,confId=i)
      shutil.copy("/content/"+filename, "/content/drive/MyDrive/MEET-U/conformations_ligands_pdb") #change directory


In [8]:
# test on known inhbitor CID2807230
show_n_conformers('C=CCN1C(=NNC1=S)CSC2=CC=CC=C2[N+](=O)[O-]','C12H12N4O2S2',3)

3 True


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
# generating conformations for all ligands
for i in inhib_tri.index:
  print(i)
  smiles = inhib_tri.loc[i]['smiles']
  formula = inhib_tri.loc[i]['formula']
  show_n_conformers(smiles, formula, 3)